# Statistical Baseline Prediction

## Notebook Configuration && Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np

from storesales.constants import (
    SUBMISSIONS_PATH,
    EXTERNAL_TRAIN_PATH,
    EXTERNAL_SAMPLE_SUBMISSION_PATH,
    EXTERNAL_TEST_PATH,
)

## Load && Prepare Data

In [113]:
def rmsle(y_true, y_pred):
    print(y_true)
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

In [4]:
train_df = pd.read_csv(EXTERNAL_TRAIN_PATH, parse_dates=["date"], index_col="id")
train_df.sort_values(by=["date", "store_nbr", "family"], inplace=True)
train_df["day_of_week"] = train_df["date"].dt.dayofweek
train_df["is_weekend"] = train_df["day_of_week"] >= 5

test_df = pd.read_csv(EXTERNAL_TEST_PATH, parse_dates=["date"])

sample_submission_df = pd.read_csv(EXTERNAL_SAMPLE_SUBMISSION_PATH, index_col="id")

# Mean Prediction

In [5]:
prediction_col_name = "window_sales_shift"

In [16]:
def predict(df: pd.DataFrame, cols: list, window: int) -> pd.DataFrame:
    df["mean_sales"] = df.groupby(cols)["sales"].rolling(window=window).mean().reset_index(level=cols, drop=True)
    # shifted_mean_sales = df.groupby(cols)["mean_sales"].shift(1)
    df[prediction_col_name] = df.groupby(cols)["mean_sales"].shift(1)
    # df.dropna(inplace=True)
    return df


def weekend_group_predict(
    df: pd.DataFrame, cols: list, window_weekday: int, window_weekend: int
) -> pd.DataFrame:
    def get_rolling_mean(group, is_weekend):
        if is_weekend:
            return group.rolling(window=window_weekend).mean()
        return group.rolling(window=window_weekday).mean()

    df["mean_sales"] = (
        df.groupby(cols)[["sales", "is_weekend"]]
        .apply(
            lambda group: get_rolling_mean(group["sales"], group["is_weekend"].iloc[0])
        )
        .reset_index(level=cols, drop=True)
    )
    df[prediction_col_name] = df.groupby(cols)["mean_sales"].shift(1)

    df.dropna(inplace=True)
    return df


def eval_prediction(df: pd.DataFrame) -> float:
    test_date_start = test_df["date"].min() - pd.Timedelta(days=17)
    test = df[df["date"] >= test_date_start]
    y_true, y_pred = test["sales"], test[prediction_col_name]
    return rmsle(y_true, y_pred)

In [116]:
df = train_df.copy()
cols = ["store_nbr", "family"]
window = 14

df["mean_sales"] = df.groupby(cols)["sales"].rolling(window=window).mean().reset_index(level=cols, drop=True)
df[prediction_col_name] = df.groupby(cols)["mean_sales"].shift(1)
df.dropna(inplace=True)

prediction_iter = iter(df[prediction_col_name])

error = df.groupby(cols)["sales"].rolling(window=window).apply(lambda x: sum(x - next(prediction_iter)))

KeyboardInterrupt: 

In [118]:
print(next(prediction_iter))

141.92357071428572


In [111]:
import pandas as pd

# Sample dataframe with values
data = [1, 2, 3, 4, 5, 6, 7]
df = pd.DataFrame(data, columns=['Values'])

# Size of the rolling window
window_size = 3

# Subtract a number from each value (for example, subtracting 1)
subtract_value = 1

# Apply rolling window and subtract
df['Windowed'] = df['Values'].rolling(window=window_size).apply(lambda x: sum(x - subtract_value))

# Drop rows where rolling operation couldn't generate a full window
df.dropna(inplace=True)

print(df)

   Values  Windowed
2       3       3.0
3       4       6.0
4       5       9.0
5       6      12.0
6       7      15.0


In [53]:
def implement_mean_grouped_prediction(df: pd.DataFrame, cols: list, window: int) -> pd.DataFrame:
    
    
    # print(f"Mean Error: {error.mean()}")

In [55]:
df = pd.DataFrame(
    {"data": [1, 2, 3, 4, 5, 6, 7], "data2": [1, 2, 3, 4, 5, 6, 7]}
)

In [102]:
df['data'].rolling(window=3).apply(lambda x: (x - 2).mean())

0    NaN
1    NaN
2    0.0
3    1.0
4    2.0
5    3.0
6    4.0
Name: data, dtype: float64

In [54]:
group_columns = ["store_nbr", "family"]
n_values_for_mean = 14

implement_mean_grouped_prediction(train_df.copy(), group_columns, n_values_for_mean)

DataError: Cannot aggregate non-numeric type: datetime64[ns]

In [11]:
group_columns = ["store_nbr", "family", "is_weekend"]
n_window_weekdays = 10
n_window_weekends = 6

predict_df = weekend_group_predict(
    train_df.copy(), group_columns, n_window_weekdays, n_window_weekends
)
eval_prediction(predict_df)

np.float64(0.4542795087195954)

In [274]:
grouped_by_is_weekend_sub_file = "grouped_by_is_weekend_submission.csv"
grouped_by_is_weekend_sub = sample_submission_df.copy()

test_prediction = predict_df[predict_df["date"] == predict_df["date"].max()]

sub_values = test_df.merge(
    test_prediction[["store_nbr", "family", prediction_col_name]],
    on=["store_nbr", "family"],
    how="left",
)

In [275]:
sub_values.head()

,index,id,date,store_nbr,family,onpromotion,window_sales_shift
0,0,3000888,2017-08-16,1,AUTOMOTIVE,0,4.8
1,1,3000889,2017-08-16,1,BABY CARE,0,0.0
2,2,3000890,2017-08-16,1,BEAUTY,2,4.6
3,3,3000891,2017-08-16,1,BEVERAGES,20,2150.2
4,4,3000892,2017-08-16,1,BOOKS,0,0.0


In [276]:
sub_values.set_index("id", inplace=True)

grouped_by_is_weekend_sub["sales"] = sub_values[prediction_col_name]

In [277]:
grouped_by_is_weekend_sub.to_csv(
    os.path.join(SUBMISSIONS_PATH, grouped_by_is_weekend_sub_file)
)

In [9]:
group_columns = ["store_nbr", "family"]
n_values_for_mean = 14

predict_df = predict(train_df.copy(), group_columns, n_values_for_mean)
eval_prediction(predict_df)

TypeError: incompatible index of inserted column with frame index

In [192]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

In [238]:
outer_cv = TimeSeriesSplit(n_splits=5, test_size=40)
inner_cv = TimeSeriesSplit(n_splits=12, test_size=40)


search_range = pd.date_range(train_df["date"].min(), test_df["date"].min(), freq="D")

# for train_idx, test_idx in outer_cv.split(search_range):
#     train_con = train_df["date"].isin(search_range[train_idx])
#     test_con = train_df["date"].isin(search_range[test_idx])
#
#     X_train = train_df[train_con].copy()
#     X_test = predict_df[test_con].copy()
#
#
#
#     y_train = predict_df[train_con]["sales"]
#     y_test = predict_df[test_con]["sales"]

## 1) Mean Prediction

In [33]:
submission_file_name = "all_period_mean_submission.csv"

In [34]:
all_period_mean_submission_df = sample_submission_df.copy()
columns = ["store_nbr", "family"]

mean_sales_per_store = train_df.groupby(columns)["sales"].mean()
prediction = test_df.merge(mean_sales_per_store, on=columns, how="left")
prediction.set_index("id", inplace=True)

all_period_mean_submission_df["sales"] = prediction["sales"]

In [35]:
submission_path = os.path.join(SUBMISSIONS_PATH, submission_file_name)
all_period_mean_submission_df.to_csv(submission_path)

## 2) Mean Sales Per Last Period

In [36]:
period_length = [7, 14, 30, 50, 70]  # days
submission_file_name = "last_period_mean_submission"

In [38]:
last_period_mean_submission_df = sample_submission_df.copy()
columns = ["store_nbr", "family"]

for period in period_length:
    date_condition = train_df["date"] > train_df["date"].max() - pd.Timedelta(
        days=period
    )
    mean_sales_per_store = train_df[date_condition].groupby(columns)["sales"].mean()
    prediction = test_df.merge(mean_sales_per_store, on=columns, how="left")
    prediction.set_index("id", inplace=True)

    last_period_mean_submission_df["sales"] = prediction["sales"]

    submission_path = os.path.join(
        SUBMISSIONS_PATH, f"{submission_file_name}_{period}_days.csv"
    )
    last_period_mean_submission_df.to_csv(submission_path)

## 3) Mean Sales For Each Day Of The Week

In [1]:
submission_file_name = "day_of_week_mean_submission"

# Facebook Prophet

In [5]:
from prophet import Prophet

## run prophet for each store - family pair

In [15]:
def run_prophet(x_train_df: pd.DataFrame, x_test_df: pd.DataFrame):
    date_range = pd.date_range(start=test_df["date"].min(), end=test_df["date"].max(), freq="D")
    future = pd.DataFrame(date_range, columns=["ds"])
    
    for (store_nbr, family), group in x_train_df.groupby(["store_nbr", "family"]):
        
        x_prophet_df = group[["date", "sales"]].copy()
        x_prophet_df.rename(columns={"date": "ds", "sales": "y"}, inplace=True)
        
        m = Prophet()
        m.fit(x_prophet_df)
        
        # Filter test_df for corresponding store_nbr and family
        test_subset = x_test_df[(x_test_df["store_nbr"] == store_nbr) & (x_test_df["family"] == family)]
        
                
        # Predict sales for future dates
        forecast = m.predict(future)
        
        # Add store_nbr and family columns back to the forecast
        forecast["store_nbr"] = store_nbr
        forecast["family"] = family
        
        # Select the relevant columns: 'ds' (date) and 'yhat' (forecasted sales)
        results.append(forecast[["ds", "yhat", "store_nbr", "family"]])

    # Concatenate all the forecast results
    forecast_df = pd.concat(results)
    
    # Merge the forecast back with the test_df
    final_df = x_test_df.merge(forecast_df, left_on=["date", "store_nbr", "family"], right_on=["ds", "store_nbr", "family"], how="left")
    
    # Drop the 'ds' column, as it's redundant after the merge
    final_df.drop(columns=["ds"], inplace=True)

    return final_df


In [16]:
run_prophet(train_df, test_df)

10:19:49 - cmdstanpy - INFO - Chain [1] start processing
10:19:49 - cmdstanpy - INFO - Chain [1] done processing


             ds
1695 2017-08-27
1696 2017-08-28
1697 2017-08-29
1698 2017-08-30
1699 2017-08-31


In [63]:
import itertools
from tqdm import tqdm
import logging

In [64]:
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

In [59]:
prophet_submission_file = "prophet_submission.csv"
prophet_submission_df = sample_submission_df.copy()

In [60]:
train_groups = train_df.groupby(["store_nbr", "family"])
test_groups = test_df.groupby(["store_nbr", "family"])

In [61]:
stores = train_df["store_nbr"].unique()
families = train_df["family"].unique()

In [65]:
for store_nbr, family in tqdm(itertools.product(stores, families), total=len(stores) * len(families)):
    train_group = train_groups.get_group((store_nbr, family))
    test_group = test_groups.get_group((store_nbr, family))
    
    test_group = test_group.rename(columns={"date": "ds"})
    
    train_group = train_group[["date", "sales"]].copy()
    train_group.rename(columns={"date": "ds", "sales": "y"}, inplace=True)
    
    m = Prophet()
    m.fit(train_group)
    
    forecast = m.predict(test_group)
    forecast.set_index("ds", inplace=True)
    test_group.set_index("ds", inplace=True)
    
    test_group["yhat"] = forecast["yhat"]
    test_group.set_index("id", inplace=True)
    
    prophet_submission_df.loc[test_group.index, "sales"] = test_group["yhat"]

100%|██████████| 1782/1782 [07:08<00:00,  4.16it/s]


In [67]:
prophet_submission_df.to_csv(os.path.join(SUBMISSIONS_PATH, prophet_submission_file))